# Chapter 8: Data Wrangling: Join, Combine, Reshape

In many applications, data can be spread across a number of files or databases
or be arranged in a form that is not easy for analysis.

This chapter focuses on tools to combine, join, and reaarrange data.

In [1]:
import pandas as pd
import numpy as np

## 8.1: Hierarchichal Indexing


The first of these tools is a feature of pandas called *Hierarchical Indexing*.

Hierarchical Indexing allows you to have multiple index *levels* on an axis. 
This allows you to work on higher dimensional data in a lower dimensional form.

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.094720
   2   -1.270117
   3    0.199623
b  1   -0.910398
   3   -1.300852
c  1   -0.501213
   2    0.938766
d  2   -0.224177
   3   -1.336491
dtype: float64

The index of this Series is an object called a MultiIndex which is stored as 
a list of n-tuples where n is the number of layers in the index

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

With a hierarchical indexed object, *partial* indexing is possible, allowing 
you to index across a single layer of the index.

In [4]:
data['b']

1   -0.910398
3   -1.300852
dtype: float64

In [5]:
data['b':'c']

b  1   -0.910398
   3   -1.300852
c  1   -0.501213
   2    0.938766
dtype: float64

In [6]:
data.loc[['b', 'd']]

b  1   -0.910398
   3   -1.300852
d  2   -0.224177
   3   -1.336491
dtype: float64

In [7]:
data.loc[:, 2]

a   -1.270117
c    0.938766
d   -0.224177
dtype: float64

In [8]:
data.loc[:, 2:3]

a  2   -1.270117
   3    0.199623
b  3   -1.300852
c  2    0.938766
d  2   -0.224177
   3   -1.336491
dtype: float64

Hierarchical indexing plays an important role in reshaping data and group
operations such as forming a pivot table

For example, you can unstack a Series into a DataFrame as follows

In [9]:
data.unstack()

,1,2,3
a,0.094720,-1.270117,0.199623
b,-0.910398,NaN,-1.300852
c,-0.501213,0.938766,NaN
d,NaN,-0.224177,-1.336491


In [10]:
data.unstack().stack()

a  1    0.094720
   2   -1.270117
   3    0.199623
b  1   -0.910398
   3   -1.300852
c  1   -0.501213
   2    0.938766
d  2   -0.224177
   3   -1.336491
dtype: float64

With a DataFrame, either axis can have a hierarchical index.

In [11]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

Each level of the hierarchical index can also have a name

In [12]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [13]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

A MultiIndex can also be created by itself and resused multiple times (since
it is unmutable)

### Reordering and Sorting Levels

Occasionally, you will need to rearrange the order of the levels on an axis
or sort the data by the values in a specific level.

The `swaplevel` method takes two level numbers or names and returns a new
object with the levels interchanged

In [14]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

`sort_index` on the other hand, sorts the data using only the values in a single
level.

In [15]:
frame.swaplevel('key1', 'key2').sort_index(level='key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistics by Level

Many descriptive and summary statistics on pandas objects have a level option
which you can specify the level you want to aggregate by

Use `groupby` to return a groupby object, which can then aggregate across the 
groups you specified

In [16]:
frame.groupby(level='key1').sum()

state  Ohio     Colorado
color Green Red    Green
key1                    
a         3   5        7
b        15  17       19

In [17]:
frame.groupby(axis=1, level='color').sum()

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame's columns

It's not unusual to want use one or more columns from DataFrame as the row 
index. 

Alternatively, you may wish to move the row index into the DataFrame's
columns.

In [18]:
frame = pd.DataFrame({'a':range(7), 'b':range(7, 0, -1),
                      'c':['one', 'one', 'one ', 'two', 'two', 'two', 'two'],
                      'd':[0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame's `set_index` function will create a new DataFrame using one or more
of its columns as the index:

In [19]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c    d      
one  0  0  7
     1  1  6
one  2  2  5
two  0  3  4
     1  4  3
     2  5  2
     3  6  1

In [20]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 Combinging and Merging Datasets


Data contained in pandas objects can be combined with the following methods:
- `pandas.merge` connects rows in DataFrames based on one or more keys. This 
    will be familliar to users of SQL or other relational databses.
- `pandas.concat` concatenates or "stackes" together object along an axis.
- The `combine_first` instance method enables splicing together overlapping
    datato fill in missing values in one object with values from another

### Database-Style DataFrame Joins

*Merge* or *join* operations combine datasets by linking row using one or more
*keys*. These operations are central to relational databases.

The *merge* function in pandas is the main entry point for using these 
algorithms on your data.

In [21]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df1


,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [22]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df2

,key,data2
0,a,0
1,b,1
2,d,2


There are multiple types of join behaviors.

The first we will look at is a *many-to-one* join.

The df1 has multiple rows labeled a and b whereas df2 only has one row for each
value in the key column.

In [23]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


If the column names are different in each objet you can specify them
sperately using `left_on` and `right_on`.

By default, join does an 'inner' join, but using the `how` parameter, you can
change how the output keys are selected

Different join types with how argument
- `'inner'` Use only the key combinations observed in both tables
- `'left'` Use only the keys observed in the left table
- `'right'` Use only the keys observed in the right table
- `'output'` Use all the key combinations observed in either tables

In [24]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


*Many-to-many* merges have well-defined, though not necessarily intuitive, 
behavior.

In [25]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data2': range(6)})
df1

,key,data2
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [26]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [27]:
pd.merge(df1, df2, on='key', how='left')

,key,data2_x,data2_y
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


Many-to-manu joins for the Cartesian product of the rows.

Since there were three 'b' rows in the left DataFrame and two in the right one, 
there are six 'b' rows in the result.

In [28]:
pd.merge(df1, df2, how='inner')

,key,data2
0,b,1
1,a,2


To merge with multiple keys, pass a list of column names

In [29]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [30]:
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [31]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


To determine which key combination will appear in the result depending on
the choice of merge method, think of the multiple keys as forming an array
of tuples to be used as a single join key.

A last issue to consider in merge operations is the treatment of overlapping 
column names.

Merge has a `suffixes` option for specifying strings to append to overlapping
names

In [32]:
pd.merge(left, right, on='key1', suffixes=['_left', '_right'])

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


merge function arguments:
- `left` 
- `right`
- `how` one of 'inner', 'outer', 'left', or 'right.
- `on` Column names to join on. Must be found in both DataFrames
- `left_on` Above but for left DataFrame
- `right_on` Above but for right DataFrame
- `left_index` Use row index in left as its join keys
- `right_index` Use row index in right as its join keys
- `sort` Sort merged data lexicographically
- `suffixes` Tuple of string values to append to column names in case of overlap
- `copy` if False, avoid copying data into resulting data structure in some
    exceptional cases
- `indicator` Adds a special column_merge that indicates the source of each
    row. Values will be 'left_only', 'right_only', or 'both'

### Merging on Index

In some cases, the merge key(s) in a DataFrame will be found in its index.

In this case, you can pass `left_index=True` or `right_index=True`

In [33]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [34]:
right1 = pd.DataFrame({'group_val': [3.5, 7]},
                       index=['a', 'b'])
right1

,group_val
a,3.5
b,7.0


In [35]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [36]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


With hier indexed data, things are more complicated as joining on index is 
implicitly a multi-key merge.

In [37]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [38]:
righth = pd.DataFrame(np.arange(12).reshape((6,2)), 
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [39]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [40]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True,
         how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


If you want to join both DataFrames by index, you can also use the `join` method

The `join` method can take a list of dataframes to join

### Concatenating Along an Axis

Another kind of data combination operation is referred to interchangeably as
concatenation, binding, or stacking.

pandas objects, having labeled axes enable you to further generalize 
concatenation.
- If the objects are indexed differently, should we combine the distinct
  elements in these axes or only use their shared components
- Do the concatenated chunks of data need too be identifiable in the resulting
  object?
- Does the 'concatenation axis' contain data that needs to be preserved?

The `concat` funcion in pandas provides a consistent way to address each of 
these concerns

Suppose we have 3 Series with no index overlap

In [41]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

By default `concat` works along axis=0, producing another Series.

Passing axis=1, the result will instead by a DataFrame

In [42]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In this case there is no overlap on the other axis, which as you can see is the
outer join of the indexes.

You can change which join is used by passing `join='inner'`

In [43]:
pd.concat([s1, s2, s3], axis=1, join='inner')

,0,1,2


You can use the `keys` parameter to create a hierarchical index where the upper
layer refers to which object the original data came from

In [44]:
result = pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [45]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


All of the same logic above applies to DataFrames

In [46]:
df1 = pd.DataFrame(np.arange(6).reshape((3,2)), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [47]:
df2 = pd.DataFrame(5 + np.arange(4).reshape((2,2)), index=['a', 'c'],
                   columns=['three', 'four'])
df2

,three,four
a,5,6
c,7,8


In [48]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

If you pass a dict of objects instead of a list, the dict's keys will be used
for the keys option

In [49]:
pd.concat({'level1':df1, 'level2':df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [50]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], 
          names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

Sometimes the row index does not contain any relevant information and all you 
care about is the order. In this case, pass `ignore_index=True`

In [51]:
pd.concat([df1, df2], ignore_index=True)

,one,two,three,four
0,0.0,1.0,NaN,NaN
1,2.0,3.0,NaN,NaN
2,4.0,5.0,NaN,NaN
3,NaN,NaN,5.0,6.0
4,NaN,NaN,7.0,8.0


concat function arguments:
- `objs` List or dict of pandas objects to be concatenated
- `axis` Axis to concatenate along
- `join` Either 'inner' or 'outer'
- `keys` Values to associate with objects being concatenated, forming a 
    hierarchical index along the concat axis.
- `level` Specific indexes to use as a hierarchical index level if keys passed
- `names` Names for created hierarchical levels if keys / levels passed
- `verify_integrity` Check new axis in concatenated object for duplicates
- `ignore_index` Do not preserve indexes

### Combining Data with Overlap

There is a nother data combination that is neither a merge or a concat.

You may have two datasets whose indexes overlap in full or part.

In [52]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [53]:
b = pd.Series(np.arange(len(a)), dtype=np.float64,
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [54]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [55]:
b.combine_first(a)

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

With dataframes, `combine_first` does the same thing column by column, so
you can think of it as "patching" missing data in the calling object with data
from the object you pass.

## 8.3: Reshaping and Pivoting

### Reshaping with Hierarchical Indexing

Hier index provides two primary actions for reshaping data:
- `stack` This "roates" or pivotes from the columns in the data to the rows
- `unstack` This pivotes from the rows into the columns

In [56]:
data = pd.DataFrame(np.arange(6).reshape((2,3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [57]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [58]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [59]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [60]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


Unstacking might introduce missing data if all of the values in the level aren't
found in each of the subgroups

In [61]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s1

a    0
b    1
c    2
d    3
dtype: int64

In [62]:
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
s2

c    4
d    5
e    6
dtype: int64

In [63]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [64]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [65]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [66]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

When you unstack in a DataFrame, the level unstacked becomes the lowest level
in the result

In [67]:
df = pd.DataFrame({'left': result, 'right': result+5},
                  columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [68]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [69]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivoting "Long" to "Wide" Format

A common way to store multiple time series in databases and CSV is in *long* or
*stacked* format.

In [70]:
data = pd.read_csv('examples/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [71]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')

In [72]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')

In [73]:
data = data.reindex(columns=columns)
data

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2
...,...,...,...
198,13324.600,-3.16,6.0
199,13141.920,-8.79,6.9
200,12925.410,0.94,8.1
201,12901.504,3.37,9.2


In [74]:
data.index = periods.to_timestamp('D', 'end').date
data

item,realgdp,infl,unemp
1959-03-31,2710.349,0.00,5.8
1959-06-30,2778.801,2.34,5.1
1959-09-30,2775.488,2.74,5.3
1959-12-31,2785.204,0.27,5.6
1960-03-31,2847.699,2.31,5.2
...,...,...,...
2008-09-30,13324.600,-3.16,6.0
2008-12-31,13141.920,-8.79,6.9
2009-03-31,12925.410,0.94,8.1
2009-06-30,12901.504,3.37,9.2


In [75]:
ldata = data.stack().reset_index().rename(columns={0: 'value', 'level_0':'date'})
ldata

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340
...,...,...,...
604,2009-06-30,infl,3.370
605,2009-06-30,unemp,9.200
606,2009-09-30,realgdp,12990.341
607,2009-09-30,infl,3.560


This is the *long* formate for multiple time series, or other data with two or 
more keys.

Each row in the table represents a single observation.

In some cases, this format may be more difficult to work with, and you may 
prefer to convert this to a DataFrame indexed by the date, and each column 
holding an item value.

The `pivot` method can do exactly that.

In [76]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2
...,...,...,...
2008-09-30,-3.16,13324.600,6.0
2008-12-31,-8.79,13141.920,6.9
2009-03-31,0.94,12925.410,8.1


The first two values for `pivot` are the columns to be used as the row and col
index, then the third is an optional value of the column used to fill the 
DataFrame.

Suppose you had two value columns that you wanted to reshape simultaneously.

In [77]:
ldata['value2'] = np.random.randn(len(ldata))

In [78]:
ldata.head()

,date,item,value,value2
0,1959-03-31,realgdp,2710.349,0.205261
1,1959-03-31,infl,0.000,2.020515
2,1959-03-31,unemp,5.800,-1.622232
3,1959-06-30,realgdp,2778.801,-0.208618
4,1959-06-30,infl,2.340,-0.575273


In [79]:
pivoted = ldata.pivot('date', 'item')
pivoted

value                     value2                    
item        infl    realgdp unemp      infl   realgdp     unemp
date                                                           
1959-03-31  0.00   2710.349   5.8  2.020515  0.205261 -1.622232
1959-06-30  2.34   2778.801   5.1 -0.575273 -0.208618  0.890549
1959-09-30  2.74   2775.488   5.3  1.448336  0.807687  2.074106
1959-12-31  0.27   2785.204   5.6 -0.409675  0.719419  0.693489
1960-03-31  2.31   2847.699   5.2 -0.097681  0.434911 -0.025654
...          ...        ...   ...       ...       ...       ...
2008-09-30 -3.16  13324.600   6.0  1.075863  1.206522 -2.014462
2008-12-31 -8.79  13141.920   6.9  0.111027  0.229748  0.131245
2009-03-31  0.94  12925.410   8.1  0.574236  0.128145 -1.015777
2009-06-30  3.37  12901.504   9.2 -0.054565 -1.529062  1.259068
2009-09-30  3.56  12990.341   9.6  0.612330 -1.908417 -0.407865

[203 rows x 6 columns]

In [80]:
pivoted['value'].head()

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


Note that pivot is equivalent to created a hierarchical index using set_index
followed by a call to unstack.

In [81]:
ldata.set_index(['date', 'item'])

value    value2
date       item                        
1959-03-31 realgdp   2710.349  0.205261
           infl         0.000  2.020515
           unemp        5.800 -1.622232
1959-06-30 realgdp   2778.801 -0.208618
           infl         2.340 -0.575273
...                       ...       ...
2009-06-30 infl         3.370 -0.054565
           unemp        9.200  1.259068
2009-09-30 realgdp  12990.341 -1.908417
           infl         3.560  0.612330
           unemp        9.600 -0.407865

[609 rows x 2 columns]

In [82]:
ldata.set_index(['date', 'item']).unstack()

value                     value2                    
item        infl    realgdp unemp      infl   realgdp     unemp
date                                                           
1959-03-31  0.00   2710.349   5.8  2.020515  0.205261 -1.622232
1959-06-30  2.34   2778.801   5.1 -0.575273 -0.208618  0.890549
1959-09-30  2.74   2775.488   5.3  1.448336  0.807687  2.074106
1959-12-31  0.27   2785.204   5.6 -0.409675  0.719419  0.693489
1960-03-31  2.31   2847.699   5.2 -0.097681  0.434911 -0.025654
...          ...        ...   ...       ...       ...       ...
2008-09-30 -3.16  13324.600   6.0  1.075863  1.206522 -2.014462
2008-12-31 -8.79  13141.920   6.9  0.111027  0.229748  0.131245
2009-03-31  0.94  12925.410   8.1  0.574236  0.128145 -1.015777
2009-06-30  3.37  12901.504   9.2 -0.054565 -1.529062  1.259068
2009-09-30  3.56  12990.341   9.6  0.612330 -1.908417 -0.407865

[203 rows x 6 columns]

### Pivoting "Wide" to "Long" Format

An inverse operation to `pivot` is `pandas.melt`.

Rather than transforming one column into many in a new DataFrame, it merges
multiple columns into one, producing a dataframe that is longer than the input.

In [83]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


The 'key' column may be a group indicator, and the other columns are data 
values.

When using `pandas.melt`, we indicate which columns are group indicators.

In [84]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [85]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [86]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [87]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


You can also melt without a group indicator

In [88]:
pd.melt(df)

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
9,C,7
